論文<br>
https://arxiv.org/abs/2304.06790<br>
<br>
GitHub<br>
https://github.com/geekyutao/inpaint-anything<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/Inpaint_Anything_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup environment

## git clone

In [ ]:
%cd /content

!git clone https://github.com/geekyutao/Inpaint-Anything.git

%cd /content/Inpaint-Anything
# Commits on Apr 24, 2023
!git checkout 6bf630535c9e3f06e01c0af63f2e206f38483c7c

## install libraries

In [ ]:
%cd /content/Inpaint-Anything

# change pytorch version
!pip uninstall torch torchvision torchaudio torchtext torchdata -y
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html --quiet

!pip uninstall yellowbrick imbalanced-learn -y
!python -m pip install -e segment_anything
!python -m pip install -r lama/requirements.txt
!python -m pip install yellowbrick imbalanced-learn
!pip install huggingface-hub

## import libraries

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

from PIL import Image
import matplotlib.pyplot as plt

# Setup pretrained models

In [ ]:
%cd /content/Inpaint-Anything

!mkdir -p pretrained_models

!wget -c https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth \
      -O ./pretrained_models/sam_vit_h_4b8939.pth

# ダウンロード不安定なためHuggingFace使用
# !curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
# !unzip big-lama.zip
# !mv big-lama ./pretrained_models
!mkdir -p pretrained_models/big-lama/models
!wget -c https://huggingface.co/camenduru/big-lama/resolve/main/big-lama/models/best.ckpt \
      -O pretrained_models/big-lama/models/best.ckpt
!wget -c https://huggingface.co/camenduru/big-lama/raw/main/big-lama/config.yaml \
      -O pretrained_models/big-lama/config.yaml

# Remove anything

In [ ]:
%cd /content/Inpaint-Anything

!python remove_anything.py \
    --input_img ./example/remove-anything/dog.jpg \
    --point_coords 200 450 \
    --point_labels 1 \
    --dilate_kernel_size 15 \
    --output_dir ./results \
    --sam_model_type "vit_h" \
    --sam_ckpt ./pretrained_models/sam_vit_h_4b8939.pth \
    --lama_config ./lama/configs/prediction/default.yaml \
    --lama_ckpt ./pretrained_models/big-lama/

In [ ]:
# show inference result
fig = plt.figure(figsize=(15, 10))

# input image
ax1 = fig.add_subplot(1, 3, 1)
plt.title('original', fontsize=16)
ax1.axis('off')
ax1.imshow( Image.open("./example/remove-anything/dog.jpg") )

ax1 = fig.add_subplot(1, 3, 2)
plt.title('mask', fontsize=16)
ax1.axis('off')
ax1.imshow( Image.open("./results/dog/with_mask_1.png") )

ax2 = fig.add_subplot(1, 3, 3)
plt.title('inpainted', fontsize=16)
ax2.axis('off')
ax2.imshow( Image.open("./results/dog/inpainted_with_mask_1.png") )

plt.show()

# Remove anything 2

In [ ]:
!wget -c https://www.pakutaso.com/shared/img/thumb/JK92_udekumu20150208091355_TP_V4.jpg \
      -O ./test01.jpg

In [ ]:
point_x = 375 #@param {type:"slider", min:0, max:800, step:1}
point_y = 120 #@param {type:"slider", min:0, max:462, step:1}

In [ ]:
img = cv2.imread('./test01.jpg')

cv2.circle(
    img,  
    (point_x, point_y), # 座標
    5,                  # 円の半径 
    (0,0,255),          # カラー
    -1                  # 太さ
    ) 

cv2_imshow(img)

In [ ]:
%cd /content/Inpaint-Anything

!python remove_anything.py \
    --input_img ./test01.jpg \
    --point_coords {point_x} {point_y} \
    --point_labels 1 \
    --dilate_kernel_size 15 \
    --output_dir ./results \
    --sam_model_type "vit_h" \
    --sam_ckpt ./pretrained_models/sam_vit_h_4b8939.pth \
    --lama_config ./lama/configs/prediction/default.yaml \
    --lama_ckpt ./pretrained_models/big-lama/

In [ ]:
# show inference result
fig = plt.figure(figsize=(15, 10))

# input image
ax1 = fig.add_subplot(1, 3, 1)
plt.title('original', fontsize=16)
ax1.axis('off')
ax1.imshow( Image.open("./test01.jpg") )

ax1 = fig.add_subplot(1, 3, 2)
plt.title('mask', fontsize=16)
ax1.axis('off')
ax1.imshow( Image.open("./results/test01/with_mask_2.png") )

ax2 = fig.add_subplot(1, 3, 3)
plt.title('inpainted', fontsize=16)
ax2.axis('off')
ax2.imshow( Image.open("./results/test01/inpainted_with_mask_2.png") )

plt.show()